<a href="https://colab.research.google.com/github/harsh-kmr/News-summarization-with-T5/blob/main/News_summarization_with_T5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import and install

In [ ]:
!pip install transformers
!pip install rouge-score
!pip install datasets 
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 50.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24954 sha256=ec27c42bd6f405689451abe9758964d7b043e3ad884a0d6a968a2b404280f318
  Stored in directory: /root/.cache/pip/wheels/9b/3d/39/09558097d3119ca0a4d462df68f22c6f3c1b345ac63a09b86e
Successfully built rouge-score
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import nltk
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM
from datasets import load_dataset
from tensorflow.keras.optimizers.schedules import PolynomialDecay
from tensorflow.keras.optimizers import Adam
import evaluate
import gdown

In [ ]:
MODEL_CHECKPOINT = "t5-small"

In [ ]:
MAX_INPUT_LENGTH = 1024
MIN_TARGET_LENGTH = 5
MAX_TARGET_LENGTH = 128
batch_size = 8
num_epochs = 3

In [ ]:
raw_datasets = load_dataset("xsum", split="train")

Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

Dataset xsum downloaded and prepared to /root/.cache/huggingface/datasets/xsum/default/1.2.0/082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71. Subsequent calls will reuse this data.


In [ ]:
raw_datasets

Dataset({
    features: ['document', 'summary', 'id'],
    num_rows: 204045
})

In [ ]:
def show(dataset, num_samples=3, seed=42):
  sample = dataset.shuffle(seed=seed).select(range(num_samples))
  for example in sample:
    print(f"\n'>> document: {example['document'][:500]}'")
    print(f"\n'>> document length: {len(example['document'].split())}'")
    print(f"'>> Summary: {example['summary']}'")
    print(f"\n'>> summary length: {len(example['summary'].split())}'")


show(raw_datasets)


'>> document: In Wales, councils are responsible for funding and overseeing schools.
But in England, Mr Osborne's plan will mean local authorities will cease to have a role in providing education.
Academies are directly funded by central government and head teachers have more freedom over admissions and to change the way the school works.
It is a significant development in the continued divergence of schools systems on either side of Offa's Dyke.
And although the Welsh Government will get extra cash to match '

'>> document length: 548'
'>> Summary: As Chancellor George Osborne announced all English state schools will become academies, the Welsh Government continues to reject the model here.'

'>> summary length: 21'

'>> document: Up to 100,000 youngsters will be eligible for half-price day tickets using The Young Persons 16-18 card from September.
It was agreed by the area's mayor Andy Burnham and Transport for Greater Manchester, and a similar scheme is being considered for the Met

In [ ]:
train_raw_dataset = raw_datasets.shuffle(seed=42).select(range(10000))
val_raw_dataset = raw_datasets.shuffle(seed=42).select(range(10000, 12000))
test_raw_dataset = raw_datasets.shuffle(seed=42).select(range(12000, 14000))

In [ ]:
type(train_raw_dataset)

datasets.arrow_dataset.Dataset

In [ ]:
show(train_raw_dataset)


'>> document: Scott Williams (leg) and Hallam Amos (shoulder) seem set to be ruled out of the tournament.
Wales defence coach Shaun Edwards says the duo are "long-term" injury concerns.
North moved to midfield when Williams departed during Wales' 28-25 win over England and told the media on Sunday he would play centre if called upon.
"If need be, I'll play centre. If need be, I'll play on the wing... that's the World Cup - you've got to adapt," said North.
Wales coach Warren Gatland hinted at the 2011 World C'

'>> document length: 371'
'>> Summary: Wing George North is prepared to switch to centre amid Wales' growing World Cup injury crisis.'

'>> summary length: 16'

'>> document: The search giant has launched a series of legal actions against the London-based firm in California, claiming four of its patents have been violated, and has also filed a separate case in the UK.
BT had been expecting such action after it launched its own case against Google in 2011.
That dispute has yet t

In [ ]:
show(test_raw_dataset)


'>> document: When it starts, passengers will be able to travel through the night on Fridays and Saturdays on the Victoria, Jubilee, Central, Northern and Piccadilly Lines.
It was due to start in September but was delayed due to pay and conditions disputes with unions.
Responding to Boris Johnson's July claim, Transport for London (TfL) said no date had been set, while unions said there were still issues to be agreed.
Mr Johnson made the announcement as he used a training simulator being used by some of the 2'

'>> document length: 388'
'>> Summary: The Night Tube service is to begin in July, London's mayor has said.'

'>> summary length: 13'

'>> document: At present, school league tables are published in January - after the application process has finished.
But from this year, the Department for Education will publish provisional GCSE results in mid-October.
Schools Minister Nick Gibb said it would provide a more "informed choice".
The plans, announced by the Department for Educatio

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)
model = TFAutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT)

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
inputs = tokenizer("I loved reading the Hunger Games!")
inputs

{'input_ids': [27, 1858, 1183, 8, 26049, 5880, 55, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
tokenizer.convert_ids_to_tokens(inputs.input_ids)

['▁I', '▁loved', '▁reading', '▁the', '▁Hunger', '▁Games', '!', '</s>']

In [ ]:
prefix = "summarize: "

In [ ]:
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=MAX_INPUT_LENGTH, truncation=True)
    
    labels = tokenizer(
        examples["summary"], max_length=MAX_TARGET_LENGTH, truncation=True
        )

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

In [ ]:
tokenized_train_dataset = train_raw_dataset.map(preprocess_function, batched=True)
tokenized_val_dataset = val_raw_dataset.map(preprocess_function, batched=True)
tokenized_test_dataset = test_raw_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

For summarization, one of the most commonly used metrics is the ROUGE score (short for Recall-Oriented Understudy for Gisting Evaluation). The basic idea behind this metric is to compare a generated summary against a set of reference summaries that are typically created by humans.

In [ ]:
generated_summary = "I absolutely loved reading the Hunger Games"
reference_summary = "I loved reading the Hunger Games"

In [ ]:
rouge_score = evaluate.load("rouge")
scores = rouge_score.compute(
    predictions=[generated_summary], references=[reference_summary]
)
scores

{'rouge1': 0.923076923076923,
 'rouge2': 0.7272727272727272,
 'rougeL': 0.923076923076923,
 'rougeLsum': 0.923076923076923}

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [ ]:
train_dataset = tokenized_train_dataset.to_tf_dataset(
    batch_size=batch_size,
    columns=["input_ids", "attention_mask", "labels"],
    shuffle=True,
    collate_fn=data_collator,
)
test_dataset = tokenized_test_dataset.to_tf_dataset(
    batch_size=batch_size,
    columns=["input_ids", "attention_mask", "labels"],
    shuffle=False,
    collate_fn=data_collator,
)
val_dataset = tokenized_val_dataset.to_tf_dataset(
    batch_size=batch_size,
    columns=["input_ids", "attention_mask", "labels"],
    shuffle=False,
    collate_fn=data_collator,
)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
num_train_steps = len(train_dataset) * num_epochs
lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5, end_learning_rate=0.0, decay_steps=num_train_steps
)

In [ ]:
opt = Adam(learning_rate=lr_scheduler)

In [ ]:
model.compile(optimizer=opt)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [ ]:
model.fit(
    train_dataset, validation_data=val_dataset, epochs=num_epochs
)

Epoch 1/3
1250/1250 [==============================] - 890s 678ms/step - loss: 2.9181 - val_loss: 2.5861
Epoch 2/3
1250/1250 [==============================] - 872s 698ms/step - loss: 2.7751 - val_loss: 2.5499
Epoch 3/3
1250/1250 [==============================] - 876s 701ms/step - loss: 2.7299 - val_loss: 2.5414


In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model=model, tokenizer=tokenizer, framework="tf")

print(test_raw_dataset[0]["document"])
summarizer(
    test_raw_dataset[0]["document"],
    min_length=MIN_TARGET_LENGTH,
    max_length=MAX_TARGET_LENGTH,
)


Stephen Paul Lynch, 34, from Shanlea Drive in the town, appeared at Antrim Magistrates Court accused of arson endangering life with intent.
A house on the Killyglen Road was badly damaged on Sunday when a car was set alight and left at the front door.
He also allegedly threatened to kill a man, and intimidated a woman and two children to leave the house.
On the day of the arson, 23 April, Mr Lynch is charged with wasting police time by making a false report of a crime.
The court heard police have CCTV footage of him crashing his own silver Peugeot into the house and then getting out of the vehicle, apparently unsteady on his feet, at about 16.40 BST on Sunday.
A police officer said the footage showed Mr Lynch setting light to a jacket and placing it in the footwell before flames engulfed the car.
He was also filmed kicking the window of the house and shouting at the property, the court heard.
There was no-one inside at the time.
The officer said Mr Lynch then phoned police and reported

[{'summary_text': 'A man has been charged with making a false report of a crime in Larne, a court has heard.'}]